*Datasets used in this example are a system of hard hexagons, simulated in the NVT thermodynamic ensemble in HOOMD-blue, for a dense fluid (phi065) and a solid (phi075)*

In [1]:
from bokeh.io import output_notebook

output_notebook()
import io
import time

import IPython.display
import numpy as np
import PIL.Image
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show
from freud import box, order, parallel

parallel.setNumThreads(4)
from util import cubeellipse, default_bokeh, local_to_global

# helper functions used in the notebook are below; you are free to disregard


def showarray(a, fmt="png"):
    """
    uses PIL to display an image rendered externally.

    Currently not used
    """
    f = io.BytesIO()
    PIL.Image.fromarray(a, mode="RGBA").save(f, fmt)
    #     PIL.Image.fromarray(a, mode='RGBA').save("out.png")
    return IPython.display.display(IPython.display.Image(data=f.getvalue(), width=600))

Loading BokehJS ...

# Hexatic Order Parameter

The hexatic order parameter measures how closely the local environment around a particle resembles perfect $k$-atic symmetry e.g. how closely the environment resembles hexagonal/hexatic symmetry if the order parameter is $k=6$. The order parameter is given by:

$$
\psi_k \left( i \right) = \frac{1}{n} \sum \limits_j^n e^{k i \phi_{ij}}
$$

where $\phi_{ij}$ is the angle between the vector $r_{ij}$ and $ \vec{\left(1, 0\right)}$

The pseudocode is given below

~~~
for each particle i:
    neighbors = nearestNeighbors(i, n):
    for each particle j in neighbors:
        r_ij = position[j] - position[i]
        psi_ij = arctan2(r_ij.y, r_ij.x)
        psi_array[i] += exp(complex(0,k*psi_ij))
~~~

An example which calculates and renders the scene for $\phi=0.65$ is shown below:

In [2]:
# create hexatic object
hex_order = order.HexOrderParameter(rmax=1.2, k=6, n=6)

# load the data
data_path = "ex_data/phi065"
box_data = np.load(f"{data_path}/box_data.npy")
pos_data = np.load(f"{data_path}/pos_data.npy")
quat_data = np.load(f"{data_path}/quat_data.npy")
n_frames = pos_data.shape[0]

# grab data from last frame
l_box = box_data[-1]
l_pos = pos_data[-1]
l_quat = quat_data[-1]
l_ang = 2 * np.arctan2(np.copy(l_quat[:, 3]), np.copy(l_quat[:, 0]))

# create box
fbox = box.Box(Lx=l_box["Lx"], Ly=l_box["Ly"], is2D=True)

# compute hexatic order for 6 nearest neighbors
start_time = time.time()
hex_order.compute(fbox, l_pos)
stop_time = time.time()
print(f"time to calc 1 frame = {stop_time-start_time}")
# get values from freud object
psi_k = hex_order.psi
avg_psi_k = np.mean(psi_k)

# render in bokeh
# vertex positions for hexagons
verts = [
    [0.537284965911771, 0.31020161970069976],
    [3.7988742065678664e-17, 0.6204032394013997],
    [-0.5372849659117709, 0.31020161970070004],
    [-0.5372849659117711, -0.31020161970069976],
    [-1.1396622619703597e-16, -0.6204032394013997],
    [0.5372849659117711, -0.3102016197006997],
]
verts = np.array(verts)
# create array of transformed positions
patches = local_to_global(verts, l_pos[:, 0:2], l_ang)
# create an array of angles relative to the average
a = np.angle(psi_k) - np.angle(avg_psi_k)
# turn into hex colors
hex_color = [cubeellipse(x) for x in a]
# plot
p = figure(title="Hexatic Order Parameter visualization")
p.patches(
    xs=patches[:, :, 0].tolist(),
    ys=patches[:, :, 1].tolist(),
    fill_color=hex_color,
    line_color="black",
)
default_bokeh(p)
show(p)

time to calc 1 frame = 0.003662109375


An example which shows $\phi=0.75$ is shown below

In [3]:
from freud import box, order

# create hexatic object
hex_order = order.HexOrderParameter(rmax=1.2, k=6, n=6)

# load the data
data_path = "ex_data/phi075"
box_data = np.load(f"{data_path}/box_data.npy")
pos_data = np.load(f"{data_path}/pos_data.npy")
quat_data = np.load(f"{data_path}/quat_data.npy")
n_frames = pos_data.shape[0]

# grab data from last frame
l_box = box_data[-1]
l_pos = pos_data[-1]
l_quat = quat_data[-1]
l_ang = 2 * np.arctan2(np.copy(l_quat[:, 3]), np.copy(l_quat[:, 0]))

# create box
fbox = box.Box(Lx=l_box["Lx"], Ly=l_box["Ly"], is2D=True)

# compute hexatic order for 6 nearest neighbors
start_time = time.time()
hex_order.compute(fbox, l_pos)
stop_time = time.time()
print(f"time to calc 1 frame = {stop_time-start_time}")
# get values from freud object
psi_k = hex_order.psi
avg_psi_k = np.mean(psi_k)

# render in bokeh
# vertex positions for hexagons
verts = [
    [0.537284965911771, 0.31020161970069976],
    [3.7988742065678664e-17, 0.6204032394013997],
    [-0.5372849659117709, 0.31020161970070004],
    [-0.5372849659117711, -0.31020161970069976],
    [-1.1396622619703597e-16, -0.6204032394013997],
    [0.5372849659117711, -0.3102016197006997],
]
verts = np.array(verts)
# create array of transformed positions
patches = local_to_global(verts, l_pos[:, 0:2], l_ang)
# create an array of angles relative to the average
a = np.angle(psi_k) - np.angle(avg_psi_k)
# turn into hex colors
hex_color = [cubeellipse(x) for x in a]
# plot
p = figure(title="Hexatic Order Parameter visualization")
p.patches(
    xs=patches[:, :, 0].tolist(),
    ys=patches[:, :, 1].tolist(),
    fill_color=hex_color,
    line_color="black",
)
default_bokeh(p)
show(p)

time to calc 1 frame = 0.005135774612426758
